To open on Google Colab [link](https://colab.research.google.com/github/RodrigoAVargasHdz/CHEM-4PB3/blob/main/Course_Notes/Week12/q_chem.ipynb)

In [ ]:
#@title External lybraries

!pip install pyscf
!pip install torchani
!pip install PyGeometry
!pip install -U pyberny
!pip install rdkit-pypi
!pip install py3Dmol
!pip install ase

<img src="https://www.researchgate.net/publication/343017500/figure/fig26/AS:996955509501965@1614704124866/1-Methods-of-computational-chemistry.jpg"
     alt="Markdown Monster icon"
     style="float: left; margin-right: 10px;" />


## Quantum chemistry

The field of computational chemistry could be split into three main research lines,
1. Molecular dynamics 
2. Quantum chemistry
3. Anything else

# (I) Molecular dynamics and Force fields
The central component of Molecular dynamics (MD) is to simulatee the movements of atoms and molecules using Newton's equations of motion. 
Here, the partciles are the nuclei, neglecting all electronic degrees of fredom, which are described using **force fields**.




Force field is a **model** that estimates the forces between atoms within molecules and also between molecules.\
*Why forces?*
$$
F(\mathbf{X}) = -\nabla E(\mathbf{X}) = M \dot{V}(t)\\
V(t) = \dot{\mathbf{X}}(t)
$$

Before the era of AI/ML, the energy function $E(\mathbf{X})$ was parametrized through the sum of variuos elements,
1. types of atoms
2. chemical bonds
3. dihedral angles
4. out-of-plane interactions
5. nonbond interactions
6. other terms (classical approximation of quantum phonomena).
Many parameter sets are empirical and some force fields use extensive fitting terms that are difficult to assign a physical interpretation

*Can we approximate $E(\mathbf{X})$ with a NN?*\
**Of course!!**

This has been one of the most promising current research lines, ([first paper](https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.98.146401), [review paper](https://doi.org/10.1021/acs.chemrev.0c01111)).



## [first NN-FF](https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.98.146401)

**Goal**: represent the total energy $E$ of the system as a sum of atomic contributions $E_i$, 
$$
E(\mathbf{X}) = \sum_i E_i
$$

Achitecture,\
<img src="https://journals.aps.org/prl/article/10.1103/PhysRevLett.98.146401/figures/2/medium"
     alt="Markdown Monster icon"
     style="float: left; margin-right: 10px;" />

1. $\{R^{\alpha}_i\}$ represent the Cartesian coordinates $\alpha$ of atom $i$.
2.$\{R^{\alpha}_i\}$  are transformed into a set of symmetry function values $\{G^{\mu}_i\}$ for each atom $i$.
3. $\{G^{\mu}_i\}$ are the input for the NN $(\{S_i\})$ to predict $\{E_i\}$.

$$
G^{1}_i = \sum_{i\neq j}^{all} e^{-\eta(R_{ij}-R_s)^2}f_c(R_{ij})\\
G^{2}_i = 2^{1-\varepsilon}\sum_{j,k\neq j}(1-λcos(\theta_{ijk}))^\varepsilon e^{-\eta(R^2_{ij}+R^2_{ik}+R^2_{jk})}f_c(R_{ij})f_c(R_{ik})f_c(R_{jk})
$$

with a little of patients, we could code this in torch 🔥! 


From the paper,
```
Compared with empirical potentials the number of DFT
calculations required to optimize the NN parameters is
rather large because of the very flexible functional form.
This, however, has the advantage that no modifications to
the NN are required if new DFT data are included. The
accuracy of the NN is limited only by that of the training
data. 
```

# Quantum models

The goal of quantum chemistry is to find the (quantum) state that describe an ensamble of particles described by the laws of quantum mechanics. 
For practical (chemistry) purpuses, our main challenge is to solve the Schr̈odinger equation (SE),
$$
\hat{H}(\{\mathbf{x}_i\}^N)\psi(\{\mathbf{x}_i\}^N) = E \;\psi(\{\mathbf{x}_i\}^N)\\
\left(\sum_i^N \nabla_i^2 + V(\{\mathbf{x}_i\}^N) \right)\psi(\{\mathbf{x}_i\}^N) = E \;\psi(\{\mathbf{x}_i\}^N),
$$
where,
1. $\psi(\{\mathbf{x}_i\}^N)$ is the $N$-particles wavefunction.
2. $\sum_i^N \nabla_i^2$ kinetic energy operator.
3. $V(\{\mathbf{x}_i\}^N)$ pontential energy operator that describes the interaction between the $N$-particles.
4. $E$ energy of the system.

Formally, quantum chemistry boilds down to solving the following problem,
$$
E_{gs} = \arg\min_{\psi_{\theta}(\{\mathbf{x}_i\}^N)} \langle \psi_{\theta}(\{\mathbf{x}_i\}^N)|\hat{H}(\{\mathbf{x}_i\}^N) | \psi_{\theta}(\{\mathbf{x}_i\}^N)\rangle
$$
one additionak constrain is $\langle \psi_{\theta}(\{\mathbf{x}_i\}^N)|| \psi_{\theta}(\{\mathbf{x}_i\}^N)\rangle = \int |\psi_{\theta}|^2 d\mathbf{X} = 1$. 

If you remember your quantum chemistry course, **position and momenta do not commute with each other**. This has significant implications as solving the SE become more challenging. 

*What about molecules?*,
The main component in the SE that describes a molecule is the pontential operator $V(\{\mathbf{x}_i\}^N)$.
$$
V(\{\mathbf{x}_i\}^N) = \frac{1}{2}\sum_{i\neq j}\frac{e^2}{|\mathbf{x}_i - \mathbf{x}_j|} + \sum_{i\ell}\frac{Z_\ell \;e}{|\mathbf{R}_\ell - \mathbf{x}_j|}
$$



## Hartree-Fock

Because $V(\{\mathbf{x}_i\}^N) \neq \sum_i V(\mathbf{x}_i))$, we cannot approximate $\psi_{\theta}(\{\mathbf{x}_i\}^N) = \prod_i \phi(\mathbf{x}_i)$.
However, the goal of Hartree-Fock is to generate a **non-interacting** electrons wavefunction. 
To achieve this solution, $V(\{\mathbf{x}_i\}^N)$ is approximated as,
$$
V(\{\mathbf{x}_i\}^N) \approx \sum_i V_{ext}(\mathbf{x}_i) + V_{Hartree}(\mathbf{x}_i) + V_{X}(\mathbf{x}_i)
$$

One-particle problem
$$
\hat{H}_{eff}(\mathbf{x})  \phi_i(\mathbf{x}) = ϵ_i\; \phi_i(\mathbf{x}),
$$
where $\langle \phi_i(\mathbf{x}) | \phi_j(\mathbf{x}) \rangle = δ_{ij}$


In general, this equations could be solved numerically (mesh). However, the **standard** procedure developed over the last decades is to use **atomic basis sets** to recast this problem into a set of lineal-algebra equations. 





### Basis sets

Every molecular orbital $(\phi_i(\mathbf{x}))$ obtained from soliving the HF equations, is $\phi_i(\mathbf{x}) = \sum_j c_j \varphi_i(\mathbf{x})$, where $\varphi_i(\mathbf{x})$ is an atomic orbital. 

For example, an electron $s$ is described by, $\varphi_s(\mathbf{x}) = \sum_\ell w_\ell e^{-\frac{(\mathbf{x}-\mathbf{R})^2}{\alpha_\ell}}$

In [ ]:
#@title PySCF + extra
import time

import pyscf
from pyscf import gto, scf, dft
from pyscf.data.nist import BOHR
from pyscf.dft.numint import eval_ao
from pyscf.geomopt.berny_solver import optimize

import matplotlib
import matplotlib.pyplot as plt
from IPython.display import display,clear_output

import numpy as np

In [ ]:
_level = 3
mol = pyscf.M(atom='''
                H  0. 0. 0.
                H  0. 0. 0.7408481486
            ''', basis='sto-3g', spin=2)  # H  0. 0. 0.7408481486

mf = scf.RKS(mol)
# S = mf.get_ovlp(mol)  # overlap matrix

mf.kernel()
# mf.grids.build(with_non0tab=True)
mf.grids.level = _level
mf.grids.build(with_non0tab=True)
Rgrid = mf.grids.coords
W = mf.grids.weights
y_ao = eval_ao(mol, mf.grids.coords)
print(y_ao.shape)

r = np.linalg.norm(Rgrid,axis=1)
for i,yao_i in enumerate(y_ao.T):
  plt.scatter(Rgrid[::10,-1],yao_i[::10],label=f'AO-{i}',s=5)

plt.vlines([0.,0.7408481486/BOHR],ymin=np.min(y_ao),ymax=np.max(y_ao),color='k',ls='--')
plt.legend()
plt.ylabel('Atomic orbital')
plt.xlabel('R')
plt.xlim(-2.5,3.5)

In [ ]:

_level = 3
mol = pyscf.M(atom='''
                Li  0. 0. 0.
            ''', basis='sto3g', spin=1)  # H  0. 0. 0.7408481486

mf = scf.RKS(mol)
# S = mf.get_ovlp(mol)  # overlap matrix

mf.kernel()
# mf.grids.build(with_non0tab=True)
mf.grids.level = _level
mf.grids.build(with_non0tab=True)
Rgrid = mf.grids.coords
W = mf.grids.weights
y_ao = eval_ao(mol, mf.grids.coords)
print(y_ao.shape)

r = np.linalg.norm(Rgrid,axis=1)
for i,yao_i in enumerate(y_ao.T):
  plt.scatter(Rgrid[::10,-1],yao_i[::10],label=f'AO-{i}',s=5)

# plt.vlines([0.],ymin=np.min(y_ao),ymax=np.max(y_ao),color='k',ls='--')
plt.legend()
plt.ylabel('Atomic orbital')
plt.xlabel('R')
plt.xlim(-5.,5.)

# Basis-set impact 
let's chose a geometry of a molecule of interest from [NIST](https://cccbdb.nist.gov/geom1x.asp)

In [ ]:
def get_HF_energy(basis_set_name):
  t = time.process_time()
  mol = pyscf.M(atom='''O	0.0000000	0.0000000	0.1164480; H	0.0000000	0.7534340	-0.4657900; H	0.0000000	-0.7534340	-0.4657900
            ''', basis=basis_set_name)

  mf = scf.RHF(mol)
  mf.kernel()
  elapsed_time = time.process_time() - t
  return mf.e_tot,elapsed_time


In [ ]:
basis_sets = ['sto3g','3-21G','4-31G','6-31G','6-311G','6-311G*','cc-pVDZ','cc-pVTZ','cc-pVQZ']
time_ = []
energy_ = []
for bs in basis_sets:
  e,t = get_HF_energy(bs) 
  energy_.append(e)
  time_.append(t)

energy_ = np.asarray(energy_)
time_ = np.asarray(time_)

In [ ]:
plt.figure(figsize=(9,5))
plt.plot(np.arange(len(basis_sets)),energy_,color='k')
cbar = plt.scatter(np.arange(len(basis_sets)),energy_,marker='s',c=time_,s=45)
cb = plt.colorbar(cbar)
cb.set_label('time [s]')
plt.xticks(np.arange(len(basis_sets)),basis_sets)
plt.xlabel('Basis Set')
plt.ylabel('Energy')
plt.tight_layout()

In [ ]:
# do the same for CH4
geom = '''C	0.0000000	0.0000000	0.0000000; H	0.6268480	0.6268480	0.6268480; H	-0.6268480	-0.6268480	0.6268480; H	-0.6268480	0.6268480	-0.6268480; H	0.6268480	-0.6268480	-0.6268480'''

## Geometry optimzation

One of the main application of quantum chemistry methods is to search for stable conformers. 
$V(\{\mathbf{x}_i\}^N)$ also depends on the location of the nuclei, $ \sum_{i\ell}\frac{Z_\ell \;e}{|\mathbf{R}_\ell - \mathbf{x}_j|}$. Therefore, $E_{gs}$ is a function of $\{\mathbf{R}_\ell\}^M$ (position of all nuclei).

By computing $\frac{\partial E_{gs}}{\partial \mathbf{R}_\ell}$, one can use gradient-based methods to modify the position of the nuclei to search for the most relaxed geometry.



In [ ]:
mol = gto.M(atom='N 0 0 0; N 0 0 1.2', basis='ccpvdz')
mf = scf.RHF(mol)

#
# geometry optimization for HF.  There are two entries to invoke the berny
# geometry optimization.
#
# method 1: import the optimize function from pyscf.geomopt.berny_solver
mol_eq = optimize(mf)
print(mol_eq.atom_coords())


# method 2: create the optimizer from Gradients class
mol_eq = mf.Gradients().optimizer(solver='berny').kernel()

In [ ]:
print('Optimize geometry')
print(mol_eq.atom_coords())

In [ ]:
bond = np.arange(0.8, 5.0, .1)
energy = []
force = []
grad_norm = []
mol = gto.Mole(atom=[['N', 0, 0, -0.4],
                     ['N', 0, 0,  0.4]],
               basis='ccpvdz')

mf_grad_scan = scf.RHF(mol).nuc_grad_method().as_scanner()
for r in reversed(bond):
    e_tot, grad = mf_grad_scan([['N', 0, 0, -r / 2],
                                ['N', 0, 0,  r / 2]])
  
    energy.append(e_tot)
    force.append(grad[0,2])
    grad_norm.append(np.linalg.norm(grad))


In [ ]:
_, (ax1,ax2) = plt.subplots(nrows=2,ncols=1,sharex=True)
ax1.plot(bond, energy[::-1])
ax1.set_ylabel('Energy')

# ax2.plot(bond, force[::-1])
ax2.plot(bond,grad_norm[::-1])
ax2.set_ylabel('Force')
ax2.set_xlabel('Bond distance')

plt.tight_layout()

# DFT (also one-particle systems + extra interactions)

Density functional theory, is by far the must successful computational tool to do insilico simulations [paper](https://aip.scitation.org/doi/10.1063/1.4704546).
Even, FF methods rely on DFT simulations to improve the accuracy of their models. 


<img src="https://aip.scitation.org/na101/home/literatum/publisher/aip/journals/content/jcp/2012/jcp.2012.136.issue-15/1.4704546/production/images/large/1.4704546.figures.f1.jpeg
 " alt="Markdown Monster icon"
     style="float: left; margin-right: 20px;" />




**Why is DFT so successful?**\
(Rodrigo's opinion)
* DFT computational scaling is in a sweeet spot. 
* Compare to other *ab-initio* methods, DFT could be scaled up to simulate larger number of atoms [paper](https://arxiv.org/ftp/arxiv/papers/2209/2209.12747.pdf). 
* There is a mathematical theorem that states the existance of $V_{XC}$, the potential that describes the interaction between electrons. 🔥
* The most sucessful numerical DFT framework looks like the HF one, (we are only missing one ingredient). 
* It works "great" for organic molecules.

<img src="https://aip.scitation.org/na101/home/literatum/publisher/aip/journals/content/jcp/2012/jcp.2012.136.issue-15/1.4704546/production/images/large/1.4704546.figures.f3.jpeg
 " alt="Markdown Monster icon"
     style="float: left; margin-right: 20px;" />




In [ ]:
# Simple example
mol = pyscf.M(
    atom = 'H 0 0 0; F 0 0 1.1',  # in Angstrom
    basis = '631g',
    symmetry = True,
)

mf = mol.KS()

mf.xc = 'b3lyp'
mf.kernel()

# Orbital energies, Mulliken population etc.
mf.analyze()

In [ ]:
def get_DFT_energy(basis_set_name):
  t = time.process_time()
  mol = pyscf.M(atom='''O	0.0000000	0.0000000	0.1164480; H	0.0000000	0.7534340	-0.4657900; H	0.0000000	-0.7534340	-0.4657900
            ''', basis=basis_set_name)
  mf = mol.KS()

  mf.xc = 'b3lyp'
  mf.kernel()

  elapsed_time = time.process_time() - t
  return mf.e_tot,elapsed_time


In [ ]:
basis_sets = ['sto3g','3-21G','4-31G','6-31G','6-311G','6-311G*','cc-pVDZ','cc-pVTZ','cc-pVQZ']
time_ = []
dft_energy_ = []
for bs in basis_sets:
  e,t = get_DFT_energy(bs) 
  dft_energy_.append(e)
  time_.append(t)

dft_energy_ = np.asarray(dft_energy_)
time_ = np.asarray(time_)

In [ ]:
plt.figure(figsize=(9,5))
plt.plot(np.arange(len(basis_sets)),energy_,color='red',label='HF',ls='--',marker='o')
plt.plot(np.arange(len(basis_sets)),dft_energy_,color='k')
cbar = plt.scatter(np.arange(len(basis_sets)),dft_energy_,marker='s',c=time_,s=45,label='B3LYP')
cb = plt.colorbar(cbar)
cb.set_label('time [s]')
plt.xticks(np.arange(len(basis_sets)),basis_sets)
plt.xlabel('Basis Set')
plt.ylabel('Energy')
plt.legend()
plt.tight_layout()

### Geometry optimization with DFT




In [ ]:
bond = np.arange(0.8, 5.0, .1)
dft_energy = []
dft_force = []
dft_grad_norm = []

mol = gto.Mole(atom=[['N', 0, 0, -0.4],
                     ['N', 0, 0,  0.4]],
               basis='ccpvdz')
mf = mol.KS()

mf.xc = 'b3lyp'
mf.kernel()

mf_grad_scan = mf.nuc_grad_method().as_scanner()
for r in reversed(bond):
    e_tot, grad = mf_grad_scan([['N', 0, 0, -r / 2],
                                ['N', 0, 0,  r / 2]])
  
    dft_energy.append(e_tot)
    dft_force.append(grad[0,2])
    dft_grad_norm.append(np.linalg.norm(grad))

In [ ]:
_, (ax1,ax2) = plt.subplots(nrows=2,ncols=1,sharex=True)
ax1.plot(bond, energy[::-1],label='HF')
ax1.plot(bond, dft_energy[::-1],label='B3LYP')
ax1.set_ylabel('Energy')

ax2.plot(bond, force[::-1],label='HF')
ax2.plot(bond, dft_force[::-1],label='B3LYP')
# ax2.plot(bond,grad_norm[::-1])
ax2.set_ylabel('Force')
ax2.set_xlabel('Bond distance')

plt.legend()
plt.tight_layout()

PySCF is flexible enough that one can define your own Exchange-Correlation functional. [Tutorial](https://pyscf.org/user/dft.html)

```python
HF_X, LDA_X = .6, .08
B88_X = 1. - HF_X - LDA_X
LYP_C = .81
VWN_C = 1. - LYP_C
mf_hf.xc = f'{HF_X:} * HF + {LDA_X:} * LDA + {B88_X:} * B88, {LYP_C:} * LYP + {VWN_C:} * VWN'
mf_hf.kernel()
mf_hf.xc = 'hf'
mf_hf.kernel()

```

# semi-empirical methods

The central component of semi-empirical methods (SEM) is to remove the computation of all integrals needed to represent the Hamiltonian and set them as parameters.

For the Hückel model, the Hamiltonian matrix elements $H_{ij}$ are parameterized in the following way,
$$
H_{ij} = \Biggl\{\begin{matrix}
 \alpha, \text{ if } i=j\\
 \beta, \text{ if } i,j \text{  are adjacent } \\
 0\\
\end{matrix}
$$

For ethylen, we only have two valence electrons,
$$
\begin{bmatrix}
\alpha_C &  \beta_{C-C}\\
\beta_{C-C}&  \alpha_C\\
\end{bmatrix}\begin{bmatrix}
 C^i_1\\C^i_2
\end{bmatrix} = E_i\begin{bmatrix}
 C^i_1\\C^i_2
\end{bmatrix}
$$


In [ ]:
alpha = 0. #parameters are set wrt to C
beta =  2.4 

H = np.array([[alpha,beta],[beta,alpha]])
e,v = np.linalg.eigh(H)
print('Eigenvalues')
print(e)
print('Eigenvectors')
print(v)


In [ ]:
#@title Torch and TorchANI
import torch
import torchani

## semi-empirical methods ++

SEM methods are pruned versions of the real quantum chemistry models grounded on physical principles where extra degrees of freedom could be parametrized or neglect to increase the computational performance.

**Why SEM are intersting?**
* They are the driving force for high throughput screening.
* More accurate than FF.
* With the rise of ML, more acccurate models can be parametrized.


J. Chem. Phys. 154, 244108 (2021)\
<img src="https://aip.scitation.org/na101/home/literatum/publisher/aip/journals/content/jcp/2021/jcp.2021.154.issue-24/5.0052857/20210625/images/large/5.0052857.figures.online.f2.jpeg
 " alt="J. Chem. Phys. 154, 244108 (2021)"
     style="float: left; margin-right: 20px;" />




In [ ]:
model = torchani.models.ANI2x(periodic_table_index=True)

coordinates = torch.tensor([[[0.03192167, 0.00638559, 0.01301679],
                             [-0.83140486, 0.39370209, -0.26395324],
                             [-0.66518241, -0.84461308, 0.20759389],
                             [0.45554739, 0.54289633, 0.81170881],
                             [0.66091919, -0.16799635, -0.91037834]]],
                           requires_grad=True)
# In periodic table, C = 6 and H = 1
species = torch.tensor([[6, 1, 1, 1, 1]])

energy = model((species, coordinates)).energies
derivative = torch.autograd.grad(energy.sum(), coordinates)[0]
force = -derivative
print(energy,force)

_, atomic_energies = model.atomic_energies((species, coordinates))

In [ ]:
bond = np.arange(0.8, 5.0, .1)
ani_energy = []
ani_force = []
ani_grad_norm = []
mol = gto.Mole(atom=[['N', 0, 0, -0.4],
                     ['N', 0, 0,  0.4]],
               basis='ccpvdz')

mf_grad_scan = scf.RHF(mol).nuc_grad_method().as_scanner()
for r in reversed(bond):
    e_tot, grad = mf_grad_scan([['N', 0, 0, -r / 2],
                                ['N', 0, 0,  r / 2]])
  
    energy.append(e_tot)
    force.append(grad[0,2])
    grad_norm.append(np.linalg.norm(grad))


### conformer search: RDKIT + TorchANI


In [ ]:
import py3Dmol
from rdkit import Chem
from rdkit.Chem import AllChem, Draw, rdMolDescriptors, rdDistGeom, rdMolTransforms, QED
from rdkit.Chem.Scaffolds.MurckoScaffold import GetScaffoldForMol
from rdkit.Chem.rdmolops import GetAdjacencyMatrix
from rdkit.Chem.Draw import IPythonConsole

import ase
import ase.optimize

In [ ]:
def get_xyz_coordinates(m_rdkit):
    xyz = Chem.MolToMolBlock(m_rdkit) # Generates a 3D conformer
    n_atoms = m_rdkit.GetNumAtoms() # total numbers of atoms

    xyz_ = []
    for l in xyz.splitlines()[4:4+m_rdkit.GetNumAtoms()]:
        l = l.split()
        xyz_.append(l[:4])

    xyz_str = '%s\n * (null), Energy   -1000.0000000\n' % (n_atoms)
    for xyzi in xyz_:
        xyzi_str = '%s     %.4f     %.4f     %.4f\n' % (
            xyzi[3], float(xyzi[0]), float(xyzi[1]), float(xyzi[2]))
        xyz_str += xyzi_str
    return xyz_str

def draw_3d(smiles, bool_add_H=True):

    m3 = AllChem.MolFromSmiles(smiles)
    if bool_add_H:
        m3 = Chem.AddHs(m3)
    AllChem.EmbedMolecule(m3, randomSeed=0xf00d)

    n_atoms = m3.GetNumAtoms()

    xyz_str = get_xyz_coordinates(m3)
    xyzview = py3Dmol.view(width=400, height=400)
    xyzview.addModel(xyz_str, 'xyz')
    xyzview.setStyle({'sphere': {'radius': 0.35}, 'stick': {'radius': 0.1}})
    xyzview.setBackgroundColor('0xeeeeee')
    xyzview.zoomTo()
    xyzview.show()


In [ ]:
m = 'OCCF'
mol = AllChem.MolFromSmiles(m)
mol_wH = Chem.AddHs(mol)
draw_3d(m)

In [ ]:
xi = []
si = []
for atom in mol_wH.GetAtoms():
  xi.append(atom.GetAtomicNum())
  si.append(atom.GetSymbol())

In [ ]:
bounds = rdDistGeom.GetMoleculeBoundsMatrix(mol_wH)
ps = rdDistGeom.ETKDGv3()
ps.randomSeed = 0xf00d
ps.SetBoundsMat(bounds)

# ps = rdDistGeom.EmbedParameters()
# ps.useExpTorsionAnglePrefs = False

ps.useBasicKnowledge = False
cids = rdDistGeom.EmbedMultipleConfs(mol_wH,1500,ps)
dists_etkdg = [rdMolTransforms.GetBondLength(conf,0,3) for conf in mol_wH.GetConformers()]

In [ ]:
def get_opt_geometry(xyz):
  molecule = ase.Atoms('OCCF', positions=xyz, calculator=model.ase())
  opt = ase.optimize.BFGS(molecule)
  opt.run(fmax=1e-6)

  coord = torch.from_numpy(molecule.get_positions()).unsqueeze(0).requires_grad_(True)
  species = torch.tensor(molecule.get_atomic_numbers(), device=device, dtype=torch.long).unsqueeze(0)
  
  return torch.from_numpy(molecule.get_positions()).unsqueeze(0).requires_grad_(True)

In [ ]:
for c in mol_wH.GetConformers():
  print(c.GetPositions())